In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import flet as ft

# --- Step 2: データの取得と保存 ---

def fetch_and_save():
    # 1. 身元を明かす (第1回・規約遵守)
    headers = {"User-Agent": "Student_musashinodaigaku_2522044"}
    
    # 2. 調査対象のURL（ここにアパホテルの検索結果URLを貼る）
    # ※日付やエリアを変えたURLを2つ用意
    urls = {
    "2026_冬コミ当日": "https://www.apahotel.com/resort/plan/search.html?hoteltype=hotel_resort&resorthotel-no=172&book-checkin=2026-12-30&...", 
    "2026_通常平日": "https://www.apahotel.com/resort/plan/search.html?hoteltype=hotel_resort&resorthotel-no=172&book-checkin=2026-01-27&..."
}
    
    
    conn = sqlite3.connect('hotel_analysis.db')
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS hotel_prices') # 再実行用に一度消す
    cursor.execute('CREATE TABLE hotel_prices (target_date TEXT, name TEXT, price INTEGER)')

    for label, url in urls.items():
        # 簡易的なエラー回避（URLが空の場合はスキップ）
        if "..." in url: continue 
        
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # アパホテルの要素抽出（クラス名はサイトに合わせて微調整が必要）
        items = soup.find_all('div', class_='p-hotelList_item')
        for item in items:
            name = item.find('h2').text.strip()
            # 価格の数字だけを抽出
            price_text = item.find('span', class_='p-hotelList_item_priceValue').text.replace(',', '')
            cursor.execute('INSERT INTO hotel_prices VALUES (?, ?, ?)', (label, name, int(price_text)))
        
        time.sleep(1.5) # サーバー負荷配慮（第1回）
    
    conn.commit()
    conn.close()